## Задание

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов.

## Решение:

В качестве основы для генерации текста используем роман Л.Н.Толстого "Анна Каренина".

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [3]:
path_to_file = '/content/Anna_Karenina.txt'
text = open(path_to_file, 'rb').read().decode(encoding='cp1251')
print(f'Длина текста: {len(text)} символов.')

Длина текста: 1881028 символов.


In [4]:
print(text[:200])

Анна Каренина
Лев Николаевич Толстой


Библиотека всемирной литературы (Эксмо)
«Анну Каренину» Толстой называл «романом широким и свободным». В основе этого определения пушкинский термин «свободн


In [5]:
# Количество уникальных символов, которые будут составлять наш словарь

vocab = sorted(set(text))
print(f'{len(vocab)} уникальных символов в тексте.')

148 уникальных символов в тексте.


In [6]:
# Создаем сопоставление между символами и их индексами 
# Преобразовываем текст из символьного в индексный вид

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [8]:
# Сформируем датасет, выделяя предсказываемый текст

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [9]:
# Перемешаем датасет и разделим его на батчи
BATCH_SIZE = 64

EPOCHS = 20

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [10]:
# Гиперпараметры необходимые в процессе обучения модели

vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

Создадим модель с помощью стэкинга LSTM блоков.

In [11]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units // 2,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units * 2,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [12]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           37888     
                                                                 
 lstm (LSTM)                 (64, None, 512)           1574912   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          6295552   
                                                                 
 lstm_2 (LSTM)               (64, None, 2048)          25174016  
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          12587008  
                                                                 
 dense (Dense)               (64, None, 148)           151700    
                                                                 
Total params: 45,821,076
Trainable params: 45,821,076
No

Обучаем модель.

In [13]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
    
model.compile(optimizer='adam', loss=loss)

In [14]:
!rm -rf ./training_checkpoints
!mkdir ./training_checkpoints
!ls ./training_checkpoints

In [16]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

In [17]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
291/291 [==============================] - 192s 592ms/step - loss: 2.8452
Epoch 2/20
291/291 [==============================] - 179s 608ms/step - loss: 2.0129
Epoch 3/20
291/291 [==============================] - 177s 600ms/step - loss: 1.5824
Epoch 4/20
291/291 [==============================] - 178s 605ms/step - loss: 1.4233
Epoch 5/20
291/291 [==============================] - 184s 627ms/step - loss: 1.3382
Epoch 6/20
291/291 [==============================] - 176s 598ms/step - loss: 1.2779
Epoch 7/20
291/291 [==============================] - 185s 627ms/step - loss: 1.2273
Epoch 8/20
291/291 [==============================] - 189s 640ms/step - loss: 1.1816
Epoch 9/20
291/291 [==============================] - 177s 603ms/step - loss: 1.1371
Epoch 10/20
291/291 [==============================] - 184s 627ms/step - loss: 1.0926
Epoch 11/20
291/291 [==============================] - 179s 609ms/step - loss: 1.0448
Epoch 12/20
291/291 [==============================] - 178s 603

In [18]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

Воспользуемся функцией с лекции для предсказания текста.

In [19]:
def generate_text(model, start_string):
    # Количество генерируемых символов
    num_generate = 700

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = 0.8

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

Сгенерируем текст.

In [20]:
start_string = "Послышались шаги и мужской голос,"
text_ = generate_text(model, start_string=start_string)
print(text_)

Послышались шаги и мужской голос, на ходу на лестницу; но он тотчас же принял участие в общей замешание и от этого напряжения.

– Да, это Алексей, – сказала она.

– Пожалуй, и некогда профессором вы убеждены, – проговорил ей Матрен Федор, и ни на минуту, по выражению лица отца и до действий ездили все мужчины в одно впечатление разговорами молодых и стариков, и манилы большое умственное развитие. Он уже был не соглашался с этим, Степан Аркадьич не нашел на нее. «Я изменяю своему положению посмотреть на него. «Я знаю, что в можно, как наши откашли!» (62, 214).

Переход от «Колпа! «Бог» он придет? – невольно красно обещает.

– Нет, молодцом. Только бы тут. Теперь идите, ми. Вот у тебя красивой женщиной (франц.).





видим значительное снижение лосса. Текст генерируется, похож на настоящий, но если вглядеться, смысл оставляет желать лучшего.